In [1]:
%env CUDA_VISIBLE_DEVICES=6

env: CUDA_VISIBLE_DEVICES=6


# Постановка задачи

Введем **операцию сравнения** двух векторов f(array_x, array_y) следующим образом:
 1. вектора поэлементно умножаются друг на друга
 2. полученные числа возводятся в квадрат и суммируются

Задан массив **data** размера **(X, Y, Z)**. Для каждого столбца **(data[x, y, :])** необходимо посчитать среднее значение от применения операции сравнения (определена выше) со всеми его соседями в окне по латерали **KxK**.

Итеративно минимизировать время работы.

<img src="images/cube.PNG" height=400, width=400> <img src="images/window.PNG" height=400, width=400>

# Код

In [2]:
import numpy as np

In [3]:
X = 2000
Y = 2000
Z = 20
K = 3
data = np.random.rand(X,Y,Z).astype('float32')

# Решение 1 (Неоптимальное)

In [ ]:
# Операция сравнения векторов:
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

<img src="images/sol1_step1.PNG" height=320, width=320> <img src="images/sol1_step2.PNG" height=300, width=300> <img src="images/sol1_step3.PNG" height=302, width=302> 

<img src="images/points.png" height=100, width=100, align="middle"> 

In [ ]:
%%time

# Переменная для хранения результатов
res = np.zeros(data.shape[0:2])
k = K // 2 + 1

for x in range(X):
    for y in range(Y):     
        # Для каждого столбца
        col = data[x, y, :]       
        accumulator = 0.0
        for x_shift in range(-k, k + 1):
            for y_shift in range(-k, k + 1):
                # Для каждого соседнего вектора в окне КxК
                if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                    # Сравнить и аккумулировать результат для столбца
                    accumulator += vectors_comparison(col, data[x + x_shift, y + y_shift])
        # Разделить на количество соседей
        x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
        y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
        neighbours_amount = x_window_length * y_window_length - 1
        res[x, y] = accumulator / neighbours_amount

# Решение 1 + Numba

## Numba

 * Компилятор для python и numpy.
 * Поддерживает некоторое подмножество операций и структур данных для работы с математическими вычислениями.
 * Существует расширение numba-scipy. 

## Вариант 1: Только декоратор

In [21]:
import numba
from numba import njit

In [22]:
X = np.int32(2000)
Y = np.int32(2000)
Z = np.int32(20)
K = np.int32(3)

In [ ]:
%%time

@njit
def vectors_comparison(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit
def get_comparison(data, X, Y, Z, K, comparison_function):
    k = K // 2 + 1
    res = np.zeros(data.shape[0:2])
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            accumulator = 0.0
            for x_shift in range(-k, k+1):
                for y_shift in range(-k, k+1):
                    if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                        accumulator += comparison_function(col, data[x + x_shift, y + y_shift])
            x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
            y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
            neighbours_amount = x_window_length * y_window_length - 1
            res[x, y] = accumulator / neighbours_amount
    return res

In [ ]:
%%time
_ = get_comparison(data, X, Y, Z, K, vectors_comparison)

In [ ]:
%%time
_ = get_comparison(data, X, Y, Z, K, vectors_comparison)

In [ ]:
%%timeit
_ = get_comparison(data, X, Y, Z, K, vectors_comparison)

## Вариант 2: Задать сигнатуры

In [ ]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]))
def vectors_comparison_sign(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
      locals={'k': numba.types.int32,
              'res': numba.types.float32[:, :],
              'col': numba.types.float32[:],
              'accumulator': numba.types.float32,
              'neighbours_amount': numba.types.int32,
              'x_window_length': numba.types.int32,
              'y_window_length': numba.types.int32})
def get_comparison_sign(data, X, Y, Z, K):
    k = K // 2 + 1
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in range(X):
        for y in range(Y):        
            col = data[x, y, :]        
            accumulator = 0.0
            for x_shift in range(-k, k+1):
                for y_shift in range(-k, k+1):
                    if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                        accumulator += vectors_comparison(col, data[x + x_shift, y + y_shift])
            x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
            y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
            neighbours_amount = x_window_length * y_window_length - 1
            res[x, y] = accumulator / neighbours_amount
    return res

In [ ]:
%%time
_ = get_comparison_sign(data, X, Y, Z, K)

In [ ]:
%%timeit
_ = get_comparison_sign(data, X, Y, Z, K)

In [19]:
from numba import prange

## Вариант 3: Использовать дополнительные параметры

In [24]:
%%time

@njit(numba.types.float32(numba.types.float32[:], 
                          numba.types.float32[:]),
      fastmath=True)
def vectors_comparison_params(vector1, vector2):
    res = np.multiply(vector1, vector2)
    res = np.sum(np.power(res, 2))
    return res

@njit(numba.types.float32[:, :](numba.types.float32[:, :, :], 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32, 
                                numba.types.int32),
      locals={'k': numba.types.int32,
              'res': numba.types.float32[:, :],
              'col': numba.types.float32[:],
              'accumulator': numba.types.float32,
              'neighbours_amount': numba.types.int32,
              'x_window_length': numba.types.int32,
              'y_window_length': numba.types.int32}, 
      parallel=True)
def get_comparison_params(data, X, Y, Z, K):
    k = K // 2 + 1
    res = np.zeros(data.shape[0:2], dtype='float32')
    for x in prange(X):
        for y in range(Y):        
            col = data[x, y, :]        
            accumulator = 0.0
            for x_shift in range(-k, k+1):
                for y_shift in range(-k, k+1):
                    if (0 <= x + x_shift < X) and (0 <= y + y_shift < Y):
                        accumulator += vectors_comparison_params(col, data[x + x_shift, y + y_shift])
            x_window_length = (min(x + x_shift, X - 1) - max(0, x - x_shift) + 1)
            y_window_length = (min(y + y_shift, Y - 1) - max(0, y - y_shift) + 1)
            neighbours_amount = x_window_length * y_window_length - 1
            res[x, y] = accumulator / neighbours_amount
    return res

CPU times: user 1.79 s, sys: 16.7 ms, total: 1.81 s
Wall time: 1.8 s


In [25]:
%%timeit
_ = get_comparison_params(data, X, Y, Z, K)

37.1 s ± 1.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Решение 2: NumPy

## Модификация 1:

<img src="images/sol2_step1.PNG" height=400, width=400>

<img src="images/sol2_step2.PNG" height=400, width=400> <img src="images/sol2_step3.PNG" height=408, width=408>


## Модификация 2:

<img src="images/sol2_step3.PNG" height=417, width=417>  <img src="images/sol2_step4.PNG" height=408, width=408>

In [ ]:
xp = np

In [ ]:
def matrices_comparison(matrix1, matrix2):
    res = xp.multiply(matrix1, matrix2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def get_comparison(data, X, Y, Z, K, comparison_function):
    k = K // 2 + 1 
    padded_data = xp.pad(data, ((0, k), (k, k), (0, 0)), constant_values=xp.nan)
    result = []      
    for x_i in range(k):
        for y_i in range(-k+1, k):
            if (x_i == 0) and (y_i <= 0):
                continue            
            # Сравнить матрицу и матрицу со сдвигом
            comparison = comparison_function(data, padded_data[x_i:x_i+X, k+y_i:k+y_i+Y])
            # Сдвинуть результат сравнения
            shifted_comparison = xp.pad(
                comparison[:X-x_i, max(0, -y_i):min(Y, Y-y_i)], 
                ((x_i, 0), (max(0, y_i), -min(0, y_i))), 
                constant_values=xp.nan
            )
            result.extend([comparison, shifted_comparison])
    return xp.nanmean(result)

In [ ]:
%%timeit
_ = get_comparison(data, X, Y, Z, K, matrices_comparison)

# Решение 2 + CuPy

 * GPU accelerator для python
 * поддерживает много функций из numpy

In [4]:
import cupy as cp
xp = cp

In [5]:
def matrices_comparison(matrix1, matrix2):
    res = xp.multiply(matrix1, matrix2)
    res = xp.sum(xp.power(res, 2), axis = 2)
    return res

def get_comparison(data, X, Y, Z, K, comparison_function):
    k = K // 2 + 1 
    padded_data = xp.pad(data, ((0, k), (k, k), (0, 0)), constant_values=xp.nan)
    result = []    
    for x_i in range(k):
        for y_i in range(-k+1, k):
            if (x_i == 0) and (y_i <= 0):
                continue            
            comparison = comparison_function(data, padded_data[x_i:x_i+X, k+y_i:k+y_i+Y])
            shifted_comparison = xp.pad(
                comparison[:X-x_i, max(0, -y_i):min(Y, Y-y_i)], 
                ((x_i, 0), (max(0, y_i), -min(0, y_i))), 
                constant_values=xp.nan
            )
            result.extend([comparison, shifted_comparison])
    return xp.nanmean(cp.asarray(result))

In [6]:
%%time
data_n = cp.asarray(data)

CPU times: user 127 ms, sys: 439 ms, total: 566 ms
Wall time: 592 ms


In [ ]:
%%time 
_ = get_comparison(data_n, X, Y, Z, K, matrices_comparison)

In [ ]:
%%timeit
_ = get_comparison(data_n, X, Y, Z, K, matrices_comparison)

# Что это и зачем?

 * Это метод для сравнения соседних трасс внутри сейсмического куба.
 * Метрики (операция сравнения) позволяют оценить, какие участки являются сложными для обучения нейрости.
 * Хотим чаще сэмплировать данные для обучения там, где данные имеют более сложную структуру.

In [7]:
%%time
_ = data_n + data_n

CPU times: user 164 ms, sys: 2.98 ms, total: 167 ms
Wall time: 165 ms


In [8]:
%%time 
_ = get_comparison(data_n, X, Y, Z, K, matrices_comparison)

CPU times: user 437 ms, sys: 3.1 ms, total: 440 ms
Wall time: 439 ms


In [18]:
%%time
data_n = data_n.astype('float64')
a = get_comparison(data_n, X, Y, Z, K, matrices_comparison)
a = cp.asnumpy(a)
cp._default_memory_pool.free_all_blocks()
cp.cuda.stream.get_current_stream().synchronize()

CPU times: user 129 ms, sys: 128 ms, total: 257 ms
Wall time: 255 ms
